##### 인스타 해시태그 크롤링 ##

## 1. 계정 n개, 상단3개의 게시물 URL만 추출하는 방법

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# 1. 웹드라이버 설정 및 인스타그램 로그인
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 인스타그램 로그인 페이지로 이동
driver.get("https://www.instagram.com/accounts/login/")
time.sleep(5)  # 페이지 로드를 기다림

# 로그인 정보 입력
username = "﻿snsdkf1234@naver.com"
password = "wel1234"

driver.find_element(By.NAME, "username").send_keys(username)
driver.find_element(By.NAME, "password").send_keys(password)
driver.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button').click()

# 로그인 후 대기 시간을 늘림
time.sleep(10)

# 2. 계정 리스트 설정
accounts = ["sh.cookkook", "seoul__nadri", "teddy_zip_", "hye_foodie_"]  # 크롤링할 계정 리스트

# 각 계정의 상단 3개 게시물 URL을 크롤링
for account_name in accounts:
    driver.get(f"https://www.instagram.com/{account_name}/")
    time.sleep(5)

    # 스크롤을 내려 모든 게시물을 로드
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

    # 게시물 URL 추출 (상단 3개만)
    post_elements = driver.find_elements(By.XPATH, '//article//a')[:3]
    post_urls = [element.get_attribute('href') for element in post_elements]

    # 결과 출력
    print(f"{account_name}: {post_urls}")

# 드라이버 종료
driver.quit()


sh.cookkook: ['https://www.instagram.com/sh.cookkook/reel/C_NSq35SLqD/', 'https://www.instagram.com/sh.cookkook/reel/C_MjKDGyot0/', 'https://www.instagram.com/sh.cookkook/reel/C_Ks1gdyYx2/']
seoul__nadri: ['https://www.instagram.com/seoul__nadri/p/C_MnuelvwpY/', 'https://www.instagram.com/seoul__nadri/p/C_MngotTvdw/', 'https://www.instagram.com/seoul__nadri/p/C_KUW7vzVpM/']
teddy_zip_: ['https://www.instagram.com/teddy_zip_/reel/C4j-yClS5Ad/', 'https://www.instagram.com/teddy_zip_/p/C0_e5VhyeaN/', 'https://www.instagram.com/teddy_zip_/reel/C_NKBRGy3fA/']


## 2. URL 기반 데이터 크롤링

In [38]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
from datetime import datetime

# 웹드라이버 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 여러 개의 인스타그램 릴스 URL 리스트
urls = ['https://www.instagram.com/sh.cookkook/reel/C_J_jvLSOzd/',
         'https://www.instagram.com/sh.cookkook/reel/C_IGZmRyUU9/', 
         'https://www.instagram.com/sh.cookkook/reel/C_HYmILyqJ3/', 
         'https://www.instagram.com/sh.cookkook/reel/C_Fn9K0yzWY/',
         'https://www.instagram.com/sh.cookkook/reel/C_E-HXXSJiZ/',
         'https://www.instagram.com/sh.cookkook/reel/C_C2rgLyOvZ/', 
         'https://www.instagram.com/sh.cookkook/reel/C_CTGyVS4da/',
         'https://www.instagram.com/sh.cookkook/reel/C_AMNBgyWrN/', 
         'https://www.instagram.com/sh.cookkook/reel/C-_n13XSbP3/',
         'https://www.instagram.com/sh.cookkook/reel/C-92qPuSbcp/',
         'https://www.instagram.com/sh.cookkook/reel/C-9DaPWyVl9/',
         'https://www.instagram.com/sh.cookkook/reel/C-7MEVjSVn4/'
        # 추가 URL
        # 다른 URL도 여기에 추가 가능
]

data_list = []

# 각 URL에서 데이터 추출
for url in urls:
    driver.get(url)
    time.sleep(10)  # 페이지 로드 대기

    # 계정명 추출
    try:
        account_name = driver.find_element(By.XPATH, '//a[contains(@class, "notranslate")]').text
    except:
        account_name = "N/A"

    # 해시태그 추출
    try:
        hashtags = [tag.text for tag in driver.find_elements(By.XPATH, '//a[contains(@href, "/explore/tags/")]')]
        hashtags = hashtags[:10]  # 상위 10개 해시태그만 가져오기
    except:
        hashtags = []

    # 본문 텍스트 추출
    try:
        post_text = driver.find_element(By.XPATH, '//div[@class="_a9zr"]').text
    except:
        post_text = ""

    # 게시 날짜 추출
    try:
        post_date_element = driver.find_element(By.XPATH, '//time')
        post_date = post_date_element.get_attribute('datetime')
        post_date = datetime.strptime(post_date, "%Y-%m-%dT%H:%M:%S.%fZ").strftime("%Y-%m-%d %H:%M:%S")  # ISO 형식을 datetime으로 변환
    except:
        post_date = "N/A"

    # 현재 크롤링 시간
    crawl_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # 데이터 저장
    data = {
        'account_name': account_name,
        'hashtags': ', '.join(hashtags),
        'post_text': post_text,
        'post_date': post_date,
        'crawl_time': crawl_time,
        'post_url': url
    }

    data_list.append(data)

# 데이터프레임으로 변환
df = pd.DataFrame(data_list)

# 결과 출력
print(df)

# CSV로 저장
df.to_csv('instagram_post_data.csv', index=False)

# 드라이버 종료
driver.quit()


     account_name likes                                           hashtags  \
0    @sh.cookkook   N/A     #쿠히치, #압구정맛집, #강남맛집, #강남술집, #이자카야, #맛집추천, #규카츠   
1    @sh.cookkook   N/A  #입분식가정집, #송파맛집, #송리단길, #송리단길맛집, #잠실맛집, #즉석떡볶이,...   
2    @sh.cookkook   N/A  #컵속애, #노량진, #노량집컵밥, #노량진맛집, #동작맛집, #맛집추천, #컵냉면...   
3    @sh.cookkook   N/A             #아리계곡, #강남맛집, #강남술집, #한식주점, #맛집추천, #백숙   
4    @sh.cookkook   N/A             #수영냉삼, #부천맛집, #냉삼, #냉삼맛집, #서초갈비, #맛집추천   
5    @sh.cookkook   N/A  #화반, #방이맛집, #잠실맛집, #송파맛집, #방이동맛집, #방이동먹자골목, #맛집추천   
6   @cafe_dasique   N/A  #데이지크, #카페데이지크, #인마이디저트박스, #성수동데이트, #데이트코스, #성...   
7    @sh.cookkook   N/A          #고래와치보, #강남맛집, #선릉맛집, #선릉술집, #강남술집, #이자카야   
8    @sh.cookkook   N/A  #샤브올데이, #호텔식샤브뷔페, #별내맛집, #샤브샤브, #별내신도시, #남양주맛집...   
9    @sh.cookkook   N/A                 #한사발포차, #닭도리탕, #떡도리탕, #안주맛집, #맛집추천   
10   @sh.cookkook   N/A  #그리노성수, #성수맛집, #뇨끼맛집, #뚝섬맛집, #성수동맛집, #성수핫플, #맛...   
11   @sh.cookkook   N/A          #이복희해장, #강남맛집, #선릉맛집, #강남술집, #해

### [이슈]
1. 개발시 크롤링을 너무 자주하면 url이 막히는 것 같음
   > 비즈니스 계정이라 5번에 1번씩 막히는 이슈는 사라졌지만, 2-3시간이 한계인듯
2. 좋아요 수는 추출이 안됨
   > 좋아요 수가 꼭 필요할까? (더 찾아봐야함)

### 검토 필요
1. 회사의 IP로 영리목적의 데이터 수집에 대한 부분 * w/법무, 컴플라이언스
2. (1번 확인 후) 외부 크롤링 자체 법적 및 보안 검토 * w/법무, 정보보호(둘 다)
3. 사내망에서 크롤링이 가능할지 * w/정보보호
   > 인스타그램 보안 사이트 막힘
   > 크롤링 웹 페이지를 폐쇄망 서버에서 뿌리고있는데, 인스타그램 URL은 볼 수가없다.
   > 활용 여부는 정보보호 협의 필요
   
==================
1. ipynb을 python script로 변경해야 스케줄러 등록이 가능함
2. 인스타 계정 ID/PW 코드 노출
   > 해당 부분 코드 수정 필요
3. for문을 통해 계정 4개를 한번하는 크롤링 -> 파일 떨구는 작업 필요
   계정 목록 accounts = ['seoul__nadri', 'sh.cookkook', 'teddy_zip_', 'hye_foodie_']
4. 8/29목 오전까지_ 웹화면 예쁘게 다듬기(서비스 배포용)
5. DF 컬럼명 한글로 변경
6. 파일 1개에 날짜 계속 쌓기
7. 인스타 Like 가리기 
8. 웹 - 회계숫자 날짜로 변경

240827_ 
1. (요청) 인스타 계정 수를 늘리고 게시물 갯수를 줄인다.
   > 계정 30개, 게시물 3개씩.. 90개 
   > (현재) 인스타 계정 4개, 게시글 12개 크롤링 = 48개
2. 인스타 크롤링하는 과정을 영상(대시보드) 형태로 보여주는게 어떤지?
   > 가시적으로 너무 좋음. 매일 아침 30분정도 보는것도 나쁘지 않을듯...
   > 화면 녹화해서 FD 전달 후 영업사원 의견취합 
3. 해주실 부분
   > 11.71.35.34:30100 port 방화벽 해제 신청
   > FD 영업사원 리뷰(해당 페이지 + 영상)
   > 인스타 계정 추가(약 30개) - 팔로잉 해주신 뒤 말씀해주세요!

### 1과 2번의 코드를 합친 것
## 인스타그램 N개 계정의 상단 3개 게시물 URL만 긁어와서 데이터를 수집하는 코드

파일 제목을 지금 날짜로 표시

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
from datetime import datetime

# 1. 웹드라이버 설정 및 인스타그램 로그인
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 인스타그램 로그인 페이지로 이동
driver.get("https://www.instagram.com/accounts/login/")
time.sleep(5)  # 페이지 로드를 기다림

# 로그인 정보 입력
username = "﻿snsdkf1234@naver.com"
password = "wel1234"

driver.find_element(By.NAME, "username").send_keys(username)
driver.find_element(By.NAME, "password").send_keys(password)
driver.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button').click()

# 로그인 후 대기 시간을 늘림
time.sleep(10)

# 2. 계정 리스트 설정
accounts = ["sh.cookkook", "seoul__nadri", "teddy_zip_", "hye_foodie_"]  # 크롤링할 계정 리스트
data_list = []

# 각 계정의 상단 3개 게시물 URL을 크롤링 및 데이터 추출
for account_name in accounts:
    driver.get(f"https://www.instagram.com/{account_name}/")
    time.sleep(5)

    # 스크롤을 내려 모든 게시물을 로드
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

    # 게시물 URL 추출 (상단 3개만)
    post_elements = driver.find_elements(By.XPATH, '//article//a')[:3]
    post_urls = [element.get_attribute('href') for element in post_elements]

    for url in post_urls:
        driver.get(url)
        time.sleep(10)  # 페이지 로드 대기

        # 계정명 추출
        try:
            account_name = driver.find_element(By.XPATH, '//a[contains(@class, "notranslate")]').text
        except:
            account_name = "N/A"

        # 해시태그 추출
        try:
            hashtags = [tag.text for tag in driver.find_elements(By.XPATH, '//a[contains(@href, "/explore/tags/")]')]
            hashtags = hashtags[:10]  # 상위 10개 해시태그만 가져오기
        except:
            hashtags = []

        # 본문 텍스트 추출
        try:
            post_text = driver.find_element(By.XPATH, '//div[@class="_a9zr"]').text
        except:
            post_text = ""

        # 게시 날짜 추출
        try:
            post_date_element = driver.find_element(By.XPATH, '//time')
            post_date = post_date_element.get_attribute('datetime')
            post_date = datetime.strptime(post_date, "%Y-%m-%dT%H:%M:%S.%fZ").strftime("%Y-%m-%d %H:%M:%S")
        except:
            post_date = "N/A"

        # 현재 크롤링 시간
        crawl_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # 데이터 저장
        data = {
            'account_name': account_name,
            'hashtags': ', '.join(hashtags),
            'post_text': post_text,
            'post_date': post_date,
            'crawl_time': crawl_time,
            'post_url': url
        }

        data_list.append(data)

# 데이터프레임으로 변환
df = pd.DataFrame(data_list)

# 결과 출력
print(df)

# 현재 날짜 가져오기
current_date = datetime.now().strftime("%Y%m%d")

# 파일 저장 경로에 'instagram_post'와 날짜 추가
df.to_csv(f'instagram_post_{current_date}.csv', index=False, encoding='utf-8-sig')

print(f"Data saved to instagram_post_{current_date}.csv")



# 드라이버 종료
driver.quit()
